In [1]:
# Imports
import sys
MIN_REQ_PYTHON = (3,6)
if sys.version_info < MIN_REQ_PYTHON:
    print('Check the Kernel->Change Kernel menu and ensure that Python 3.6')
    print('or later is selected as the active kernel.')
    sys.exit("Python %s.%s or later is required.\n" % MIN_REQ_PYTHON)


from IPython.display import display
import pandas as pd

import msticpy.sectools as sectools
import msticpy.nbtools as mas
from msticpy.sectools import VTLookup, IoCExtract
from msticpy.sectools.geoip import GeoLiteLookup, IPStackLookup
from msticpy.nbtools.entityschema import IpAddress, GeoLocation
from msticpy.nbtools.foliummap import FoliumMap

In [ ]:
# OPTION 1 : import the key api named 'vt' from the keyring manager
# below the python cli (to execute once) create the api key : 
# >>> import keyring
# >>> import readline
# >>> keyring.set_password(service_name="vt",username="None",password="xxx")
# >>> readline.clear_history()
import keyring
keyring.get_keyring()
vt_apikey = keyring.get_credential("vt", "None")

In [2]:
# OPTION 2 : import the key api named from the environment variable 
import os
vt_apikey = os.environ['VT_API_KEY']

In [3]:
# Create an instance of the class
vt_lookup = sectools.VTLookup(vt_apikey, verbosity=2)

In [4]:
# How these map to VT lookup types
VTLookup._VT_TYPE_MAP

{'ipv4': 'ip-address',
 'dns': 'domain',
 'url': 'url',
 'md5_hash': 'file',
 'sha1_hash': 'file',
 'sh256_hash': 'file'}

In [5]:
# Load a CSV to build the iplist DF, aka the public IP list to enrich
# CSV format is N columns with at least one column named 'IP' 
from tkinter import Tk
from tkinter.filedialog import askopenfilename

Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
filename = askopenfilename() # show an "Open" dialog box and return the path to the selected file
print(filename)

iplist = pd.read_csv(filename)
iplist.drop_duplicates(subset='IP',inplace=True)
iplist

/home/jomivz/forum/pubip3.csv


,IP,port,CVE,CVSS,confidence
0,13.94.225.221,443,CVE-2013-4352,4.3,LOW
43,35.80.207.200,443,CVE-2017-9798,7.5,LOW
63,18.195.169.49,443,CVE-2020-1927,6.1,LOW
83,168.87.23.46,80,CVE-2017-13099,5.9,LOW


In [6]:
# Check the reputation of the iplist in VirusTotal
ioc_extractor = IoCExtract()
vt_lookup = VTLookup(vt_apikey, verbosity=2)
output_df = ioc_extractor.extract(data=iplist, 
                                  columns=['IP'], 
                                  ioc_types=vt_lookup.supported_ioc_types)
output_df

,IoCType,Observable,SourceIndex,Input
0,ipv4,13.94.225.221,0,13.94.225.221
1,ipv4,35.80.207.200,43,35.80.207.200
2,ipv4,18.195.169.49,63,18.195.169.49
3,ipv4,168.87.23.46,83,168.87.23.46


In [7]:
from past.builtins import xrange
vt_lookup = VTLookup(vt_apikey, verbosity=2)
vt_res = [[]]

# Submit a subset of the found IoCs (ignore dns since a lot of )
vt_results = vt_lookup.lookup_iocs(data=output_df, 
                                   type_col='IoCType', 
                                   src_col='Observable')

#clean the DF, deleting uninteresting cols
vt_res = vt_results.drop(columns=['IoCType', 'Status', 'ResponseCode', 'RawResponse', 'Resource', 'SourceIndex', 'VerboseMsg', 'Resource', 'ScanId', 'Permalink', 'MD5', 'SHA1', 'SHA256', 'ResolvedIPs'])


Submitting observables: "13.94.225.221", type "ipv4" to VT. (Source index {idx})
Error in response submitting observables: '13.94.225.221', type 'ipv4'http status is 403. Response: None (Source index 0
Submitting observables: "35.80.207.200", type "ipv4" to VT. (Source index {idx})
Error in response submitting observables: '35.80.207.200', type 'ipv4'http status is 403. Response: None (Source index 43
Submitting observables: "18.195.169.49", type "ipv4" to VT. (Source index {idx})
Error in response submitting observables: '18.195.169.49', type 'ipv4'http status is 403. Response: None (Source index 63
Submitting observables: "168.87.23.46", type "ipv4" to VT. (Source index {idx})
Error in response submitting observables: '168.87.23.46', type 'ipv4'http status is 403. Response: None (Source index 83
Submission complete. 4 responses from 4 input rows


In [8]:
iplocation = GeoLiteLookup()
vt_res = iplocation.df_lookup_ip(vt_res,'Observable')
display(vt_res)

Latest local Maxmind City Database present is older than 30 days. Attempting to download new database to /home/jomivz/.msticpy/GeoLite2


/home/jomivz/.local/lib/python3.9/site-packages/msticpy/sectools/geoip.py:560: UserWarning: GeoIpLookup: DB download failed
  warnings.warn(
/home/jomivz/.local/lib/python3.9/site-packages/msticpy/sectools/geoip.py:560: UserWarning: GeoIpLookup: Continuing with cached database. Results may inaccurate.
  warnings.warn(


,Observable,Positives,ResolvedDomains,DetectedUrls,CountryCode,CountryName,State,City,Longitude,Latitude,TimeGenerated,Type,IpAddress
0,13.94.225.221,NaN,NaN,NaN,NL,Netherlands,North Holland,Amsterdam,4.8975,52.3759,2022-03-12 11:07:10.877938,geolocation,13.94.225.221
1,35.80.207.200,NaN,NaN,NaN,US,United States,Oregon,Boardman,-119.7143,45.8491,2022-03-12 11:07:10.879361,geolocation,35.80.207.200
2,18.195.169.49,NaN,NaN,NaN,DE,Germany,Hesse,Frankfurt am Main,8.6843,50.1188,2022-03-12 11:07:10.879794,geolocation,18.195.169.49
3,168.87.23.46,NaN,NaN,NaN,US,United States,NaN,NaN,-97.8220,37.7510,2022-03-12 11:07:10.881089,geolocation,168.87.23.46


In [9]:
folium_map = FoliumMap(location=[vt_res.Latitude.mean(), vt_res.Longitude.mean()], zoom_start=1)
# Set Icon properties to display
icon_props = {"color": "green"}

def create_ip_entity(row):
    ip_ent = IpAddress(Address=row["Observable"])
    geo_loc = create_geo_entity(row)
    ip_ent.Location = geo_loc
    return ip_ent

def create_geo_entity(row):
    # get subset of fields for GeoLocation
    loc_props = row[["CountryCode", "CountryName","State", "City", "Longitude", "Latitude"]]
    geo_loc = GeoLocation(**loc_props.to_dict())
    return geo_loc

# Set additional properties to display in tooltips
# in AdditionalData property of IpAddress entities
geo_locs = list(vt_res.apply(create_geo_entity, axis=1).values)
ip_ents = list(vt_res.apply(create_ip_entity, axis=1).values)
folium_map.add_ip_cluster(ip_entities=ip_ents, **icon_props)

# display the map (or just have folium_map as the last statement
# of the cell - Jupyter will use display() automatically)
display(folium_map)
#folium_map.save('result.html')
#os.system('explorer.exe result.html')